# Задача 6
___
Расширим возможности калькулятора, который вы делали в первом модуле, и модифицируем логику обработки выражений. Теперь будем собирать возникающие исключения с помощью системы логирования. 

Калькулятор должен поддерживать четыре операции: сложение (+), умножение (×), вычитание (−), деление (÷), определённые с целыми числами и числами с плавающей точкой, а также должен быть толерантен к пробелам, то есть между операндами и числами может быть неограниченное число пробелов.

Для обработки выражений реализуйте функциональный подход: создайте функцию для каждой операции и используйте её как объект.

Как работает калькулятор
Вы указываете имя файла, содержащего выражения, которые нужно вычислить. Одно выражение — одна строка.

Если в выражении есть ошибки, калькулятор запоминает номера строк, в которых они встретились, и выводит ошибки в консоль и файл. 

Калькулятору также необходимо вывести номер строки с результатами вычисления и результат вычисления. В файл logs.log необходимо вывести лог ошибок, в котором также записать номер строки, в котором встретилась ошибка, и её тип. 

### Входные данные
```python
exprs.txt

2 + 3

-2 -3

-5,2   * 4

-5.2 *4

a+ 5
```
### Вывод программы
```python
results.txt
1 5.0
2 -5.0
4 -20.8
logs.log
2023-08-12 14:08:12.806 | ERROR    | __main__:<module>:69 - Line #3: could not convert string to float: '-5,2'
2023-08-12 14:08:12.806 | ERROR    | __main__:<module>:69 - Line #5: Пустая строка
2023-08-12 14:08:12.806 | ERROR    | __main__:<module>:69 - Line #6: could not convert string to float: 'a'
```
___
# Решение
(так как мы продолжаем расширять функциональность калькулятора, то объектно-ориентированное решение может быть предпочтительнее)


In [1]:
from loguru import logger
import re

# Настройка логирования
logger.add("task6.log", format="{time} | {level} | {message}", level="ERROR")

class Calculator:
    def __init__(self):
        self.operations = {
            '+': self.add,
            '-': self.subtract,
            '*': self.multiply,
            '/': self.divide
        }

    def add(self, a, b):
        return a + b

    def subtract(self, a, b):
        return a - b

    def multiply(self, a, b):
        return a * b

    def divide(self, a, b):
        if b == 0:
            raise ValueError("Division by zero")
        return a / b

    def evaluate_expression(self, expression):
        expression = expression.replace(' ', '')
        match = re.match(r'(-?\d+(\.\d+)?)([+\-*/])(-?\d+(\.\d+)?)', expression)
        if not match:
            raise ValueError(f"Invalid expression: {expression}")
        
        a, operator, b = float(match.group(1)), match.group(3), float(match.group(4))
        return self.operations[operator](a, b)

class ExpressionEvaluator:
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        self.calculator = Calculator()

    def validate_expression(self, expression):
        # Проверка на наличие недопустимых символов
        if re.search(r'[a-zA-Z]', expression):
            raise ValueError("could not convert string to float:")
        
        # Проверка на корректность выражения
        expression = expression.replace(' ', '')
        if not re.match(r'(-?\d+(\.\d+)?[+\-*/]-?\d+(\.\d+)?)', expression):
            raise ValueError(f"Invalid expression: {expression}")

    def process_expressions(self):
        with open(self.input_file, 'r') as file:
            lines = file.readlines()
        
        results = []
        for i, line in enumerate(lines, start=1):
            try:
                self.validate_expression(line.strip())
                result = self.calculator.evaluate_expression(line.strip())
                results.append(f"{i} {result:.1f}")
            except Exception as e:
                logger.error(f"Line #{i}: {e}")
        
        with open(self.output_file, 'w') as file:
            for result in results:
                file.write(result + '\n')

def main():
    input_file = "exprs.txt"
    output_file = "results.txt"
    
    evaluator = ExpressionEvaluator(input_file, output_file)
    evaluator.process_expressions()

if __name__ == "__main__":
    main()

2025-01-22 15:42:43.225 | ERROR    | __main__:process_expressions:66 - Line #3: Invalid expression: -5,2*4
2025-01-22 15:42:43.226 | ERROR    | __main__:process_expressions:66 - Line #5: could not convert string to float:
